# Configurações

In [1]:
import pandas as pd 
import geopandas as gpd
import os
import re

In [2]:
PATH_JSON = "json/"
PATH_LATLONG = "latlong/"
PATH_TO_MAKE = PATH_LATLONG + "to_make/"
PATH_GEOJSON = "geojson/"
PATH_RAW_GEOJSON = PATH_LATLONG + PATH_GEOJSON
PATH_DE_PARA = PATH_RAW_GEOJSON + "de_para/"

# De Para

Protótipo de script para alterar o nome das colunas e posteriormente realizar modificações adicionais.
Por enquanto para funcionar é preciso selecionar os arquivos a serem utilizados.

In [9]:
files_de_para = list(os.listdir(PATH_DE_PARA))
files_de_geojson = list(os.listdir(PATH_RAW_GEOJSON))

print(PATH_DE_PARA)
print(files_de_para)
print(files_de_geojson)

latlong/geojson/de_para/
['metadados_municipios_pi.csv', 'metadados_regioes_intermediarias_pi.csv', 'metadados_territorios_de_desenvolvimento_pi.csv', 'metadados_regioes_imediatas_pi.csv', 'metadados_sedes_municipais_pi.csv', 'metadados_microrregioes_pi.csv', 'metadados_mesorregioes_pi.csv']
['sedes_municipais_pi.geojson', 'microrregioes_pi.geojson', 'municipios_pi.geojson', 'territorios_desenvolvimento_pi.geojson', 'regioes_intermediarias_pi.geojson', 'regioes_imediatas_pi.geojson', 'mesorregioes_pi.geojson', 'de_para']


In [6]:
file_name_de_para = files_de_para[4]
print(f"arquivo: {file_name_de_para}")
df = pd.read_csv(PATH_DE_PARA + file_name_de_para)
df

arquivo: metadados_sedes_municipais_pi.csv


,nome_original,nome_publico,descricao
0,name,Nome da Sede Municipal,Nome da Sede Municipal
1,geocodigo_,GeoCodigo,GeoCodigo(Não tornar Público)
2,nome_munic,Nome do Município,Nome do Município
3,uf,UF,UF(Não tornar público)
4,geometry,Geometry,Geometry(Não tornar público)


In [7]:
de_para_dict = {}
# each row is a de_para "nome_original" -> "nome_publico
for index, row in df.iterrows():
    de_para_dict[row['nome_original']] = row['nome_publico']
print(de_para_dict)
de_para_dict

{'name': 'Nome da Sede Municipal', 'geocodigo_': 'GeoCodigo', 'nome_munic': 'Nome do Município', 'uf': 'UF', 'geometry': 'Geometry'}


{'name': 'Nome da Sede Municipal',
 'geocodigo_': 'GeoCodigo',
 'nome_munic': 'Nome do Município',
 'uf': 'UF',
 'geometry': 'Geometry'}

In [8]:
file_name_de_geojson = files_de_geojson[0]
print(file_name_de_geojson)
# read geojson
gdf = gpd.read_file(PATH_RAW_GEOJSON + file_name_de_geojson)
#gdf.rename(columns=de_para_dict, inplace=True)
# gdf to dict
gdf_dict = gdf.to_dict()
# replace keys

sedes_municipais_pi.geojson


In [104]:
# export gdf to geojson
gdf.to_file(PATH_RAW_GEOJSON + file_name_de_geojson, driver='GeoJSON')

## Criar geojson

In [50]:
files = os.listdir(PATH_LATLONG)

In [51]:
print(files)

['Localização das Startups - Longitude e Latitude.csv', 'to_make', 'geojson', 'espacos_de_inovacao_lat_lon_pi.csv']


In [52]:
problem_with = []

In [53]:
def get_file(files):
    for index in range(len(files)):
        yield files[index], index

In [54]:
def compile_into_one(df, column_regex_pattern):
    columns_in_df = list(df.columns)
    columns_to_compile = []
    for column in columns_in_df:
        # column match the regex pattern
        if re.match(column_regex_pattern, column):
            columns_to_compile.append(column)
    df[column_regex_pattern] = df[columns_to_compile].apply(lambda x: ', '.join(x.dropna().astype(str)), axis=1)
    df.drop(columns=columns_to_compile, inplace=True)
    return df

In [55]:
def compile_type(df, file_index):
    if file_index == 0:
        df = compile_into_one(_, "tipo_prod")
        df = compile_into_one(df, "metrica")
        df = compile_into_one(df, "prod_animal")
        df = compile_into_one(df, "valor_prod")
    elif file_index == 1:
        df = compile_into_one(_, "tipo")
        df = compile_into_one(df, "producao")
        df = compile_into_one(df, "valor_mil_reais")
    elif file_index == 2 or file_index == 4 or file_index == 8 or file_index == 9 or file_index == 10 or file_index == 11 or file_index == 13 or file_index == 15 or file_index == 16 or file_index == 17 or file_index == 18:
        df = compile_into_one(_, "curso")
        df = compile_into_one(df, "steam")
        df = compile_into_one(df, "vagas")
        df = compile_into_one(df, "cargaHoraria")
    elif file_index == 6:
        df = compile_into_one(_, "lav_permanente")
        df = compile_into_one(df, "Area_destinada_colheita_Hec")
        df = compile_into_one(df, "area_colhida_Hec")
        df = compile_into_one(df, "qtd_produzida_Toneladas")
        df = compile_into_one(df, "Valor_da_produção_Mil_Reais")
    elif file_index == 7:
        df = compile_into_one(_, "TipoRebanho")
        df = compile_into_one(df, "Efetivo")
    elif file_index == 12:
        df = compile_into_one(_, "lav_temporária")
        df = compile_into_one(df, "Area_destinada_colheita_Hec")
        df = compile_into_one(df, "area_colhida_Hec")
        df = compile_into_one(df, "qtd_produzida_Toneladas")
        df = compile_into_one(df, "Valor_da_produção_Mil_Reais")
    return df

In [56]:
def parse_city_name(_, city_column):
    _[city_column] = _[city_column].str.upper()
    _[city_column] = _[city_column].str.replace(" - PI", "")
    # remove accents
    _[city_column] = _[city_column].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    # trim whitespaces
    _[city_column] = _[city_column].str.strip()
    _[city_column] = _[city_column].str.replace("  ", " ")
    return _

def agg_top_value(row):
    return row.value_counts().index[0]

In [61]:
for file_path, index in get_file(files):
    if file_path.endswith(".csv"):
        print(file_path, index)
        _ = pd.read_csv(PATH_LATLONG + file_path, sep=",")
        print("---")
        print(list(_.columns))
        print("---")
        #_ = compile_type(_, index)
        print(_.info())
        if file_path == "espacos_de_inovacao_lat_lon_pi.csv":
            #_.drop(columns=["estado", "pais"], inplace=True)
            _.columns = ["Nome do HUB", "Municipio", "Latitude", "Longitude"]
        if file_path == "Localização das Startups - Longitude e Latitude.csv":
            #_.drop(columns=["Unnamed: 0", "uf"], inplace=True)
            _.columns = ["Nome da Startup", "CNPJ", "Municipio", "Ciclo", "Segmento", "Status do Ciclo", "Ano de inicio das atividades", "Maturidade", "Latitude", "Longitude"]
        if file_path == "producao_extracao_vegetal_lat_lon_pi.csv":
            print(1)
            _ = parse_city_name(_, "municipio")
            _ = _.groupby(["municipio", "latitude", "longitude"]).agg({
                "quantidade_produzida_na_extracao_vegetal": "sum",
                "tipo_de_produto_extrativo": agg_top_value,
            }).reset_index()
            _.columns = ["Municipio", "Latitude", "Longitude", "Total quantidade produzida na extração", "Principal tipo"]
        elif file_path == "efetivos_rebanhos_pi_lat_lon.csv":
            print(2)
            _ = parse_city_name(_, "municipio")
            _ = _.groupby(["municipio", "latitude", "longitude"]).agg({
                "tipo_de_rebanho": agg_top_value,
            }).reset_index()
            _.columns = ["Municipio", "Latitude", "Longitude", "Principal tipo de rebanho"]
        elif file_path == "producao_origem_animal_lat_lon_pi.csv":
            print(3)
            _ = parse_city_name(_, "municipio")
            _ = _.groupby(["municipio", "latitude", "longitude"]).agg({
                "tipo_de_produto_de_origem_animal": agg_top_value,
            }).reset_index()
            _.columns = ["Municipio", "Latitude", "Longitude", "Principal tipo de produto de origem animal"]
        # export to csv
        print(_.info())
        _.to_csv(PATH_LATLONG + file_path, index=False, sep=",", encoding="utf-8")

Localização das Startups - Longitude e Latitude.csv 0
---
['Nome da Startup', 'CNPJ', 'Municipio', 'Ciclo', 'Segmento', 'Status do Ciclo', 'Ano de inicio das atividades', 'Maturidade', 'Latitude', 'Longitude']
---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Nome da Startup               119 non-null    object 
 1   CNPJ                          119 non-null    object 
 2   Municipio                     119 non-null    object 
 3   Ciclo                         119 non-null    int64  
 4   Segmento                      116 non-null    object 
 5   Status do Ciclo               119 non-null    object 
 6   Ano de inicio das atividades  72 non-null     float64
 7   Maturidade                    119 non-null    object 
 8   Latitude                      119 non-null    float64
 9   Longitude                  

In [62]:
_

,Nome do HUB,Municipio,Latitude,Longitude
0,HUB De Hidrogênio Verde,Parnaíba,-2.903360,-41.778171
1,Tech Export HUB,Parnaíba,-2.903360,-41.778171
2,HUB Investe Piauí,Teresina,-5.086342,-42.805270


In [63]:
for filename in files:
    if filename.endswith(".csv"):
        print(filename)
        try:
            df = pd.read_csv(PATH_LATLONG+filename, sep=",")
        except:
            print(f"!!! problem with {filename} !!!")
            problem_with.append(filename)
            continue
        print(df.info())
        # transfrom df into geojson with latitude as column ["lat"] and longitude as column ["long"]
        try:
            gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))
        except:
            try:
                gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))
            except Exception as e:
                print(e)
                print(f"!!! problem with {filename} !!!")
                print(df.columns)
                problem_with.append(filename)
                raise e
        # save geojson
        gdf.to_file(PATH_LATLONG+filename.replace(".csv", ".geojson"), driver="GeoJSON")

Localização das Startups - Longitude e Latitude.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Nome da Startup               119 non-null    object 
 1   CNPJ                          119 non-null    object 
 2   Municipio                     119 non-null    object 
 3   Ciclo                         119 non-null    int64  
 4   Segmento                      116 non-null    object 
 5   Status do Ciclo               119 non-null    object 
 6   Ano de inicio das atividades  72 non-null     float64
 7   Maturidade                    119 non-null    object 
 8   Latitude                      119 non-null    float64
 9   Longitude                     119 non-null    float64
dtypes: float64(3), int64(1), object(6)
memory usage: 9.4+ KB
None
espacos_de_inovacao_lat_lon_pi.csv
<class 'pandas.core.frame.DataFra

In [141]:
problem_with

['producao_aquicultura_lat_lon_pi.csv',
 'producao_aquicultura_pi.csv',
 'producao_extracao_vegetal_lat_lon_pi.csv',
 'producao_extracao_vegetal_pi.csv',
 'efetivos_rebanhos_pi_lat_lon.csv',
 'producao_origem_animal_lat_lon_pi.csv',
 'efetivos_de_rebanhos_pi.csv',
 'producao_de_origem_animal_pi.csv',
 'silvicultura.csv',
 'producao_aquicultura_lat_lon_pi.csv',
 'producao_aquicultura_pi.csv',
 'producao_aquicultura_pi.csv',
 'producao_extracao_vegetal_pi.csv',
 'efetivos_de_rebanhos_pi.csv',
 'producao_de_origem_animal_pi.csv',
 'silvicultura.csv']

# Criar estatística e tabela final

Protótipo de script para reduzir a granularidade dos dados e criar indicadores para ser utilizado para criação de geojson's.

Este script também adiciona latitude e longitude a partir do nome do município e do conjunto de dados de sedes municipais.

In [11]:
files_to_make = os.listdir(PATH_TO_MAKE)
files_to_make

['volume_exportacao_mineracao_10_anos.csv',
 'negocios_pi.csv',
 'empresas_do_setor_ativas.csv',
 'extracao_mineral.csv',
 'sedes_municipais.json',
 'final']

Base de sedes municipais com lat e long.

In [14]:
repo_geodf = gpd.read_file("latlong/to_make/sedes_municipais.json")
repo_geodf = parse_city_name(repo_geodf, "name")
repo_geodf.rename(columns={"name": "Município"}, inplace=True)
repo_geodf

,Município,geocodigo_,nome_munic,uf,geometry
0,ACAUA,2200053,ACAUÃ,pi,MULTIPOINT (-41.08364 -8.22079)
1,BETANIA DO PIAUI,2201739,BETÂNIA DO PIAUÍ,pi,MULTIPOINT (-40.79662 -8.14567)
2,PAULISTANA,2207801,PAULISTANA,pi,MULTIPOINT (-41.14710 -8.14144)
3,AGRICOLANDIA,2200103,AGRICOLÂNDIA,pi,MULTIPOINT (-42.65918 -5.79550)
4,AGUA BRANCA,2200202,ÁGUA BRANCA,pi,MULTIPOINT (-42.63513 -5.89327)
...,...,...,...,...,...
218,TANQUE DO PIAUI,2210979,TANQUE DO PIAUÍ,pi,MULTIPOINT (-42.28390 -6.59635)
219,VARZEA GRANDE,2211407,VÁRZEA GRANDE,pi,MULTIPOINT (-42.25310 -6.54295)
220,UNIAO,2211100,UNIÃO,pi,MULTIPOINT (-42.86293 -4.58602)
221,URUCUI,2211209,URUÇUÍ,pi,MULTIPOINT (-44.56026 -7.22950)


Funções auxiliares.

In [12]:
def parse_city_name(_, city_column):
    _[city_column] = _[city_column].str.upper()
    _[city_column] = _[city_column].str.replace(" - PI", "")
    # remove accents
    _[city_column] = _[city_column].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    # trim whitespaces
    _[city_column] = _[city_column].str.strip()
    _[city_column] = _[city_column].str.replace("  ", " ")
    return _

def agg_top_value(row):
    return row.value_counts().index[0]


Conjunto de municípios que precisam de latitude e longitude.

In [17]:
lat_long_repo = set()

Busca do arquivo a ser modificado, por index.

In [ ]:
index_file = 3
print(files_to_make[index_file])

Conjunto de modificações a serem realizadas por index do arquivo.

In [329]:
if files_to_make[index_file].endswith(".csv"):
    _ = pd.read_csv(PATH_TO_MAKE + files_to_make[index_file], sep=",")
    

    if index_file == 0:
        _ = parse_city_name(_, "municipio")
        _.drop(columns=["uf_do_municipio", "codigo_sh4", "ano"], inplace=True)
        _ = _.groupby("municipio").agg({
            "pais_de_exportacao": agg_top_value,
            "descricao_sh4": agg_top_value,
            "valor_fob_(us$)": "mean",
            "quilograma_liquido": "mean"
        }).reset_index()
        _["valor_fob_(us$)"] = _.apply(lambda x: round(x["valor_fob_(us$)"], 2), axis=1)
        _["quilograma_liquido"] = _.apply(lambda x: round(x["quilograma_liquido"], 2), axis=1)
        for city in _.municipio:
            lat_long_repo.add(city)
        _.columns = ["Município", "Principal país de exportação", "Principal produto", "Valor FOB (US$) em media", "Quilograma líquido em média"]
        _ = pd.merge(repo_geodf, _, on="Município", how="right")
        _

    elif index_file == 1:
        print(_.info())
        _ = parse_city_name(_, "municipio")
        _ = _.groupby("municipio").agg({
            "hash_chave": "count",
            "desc_atividade": agg_top_value,
            "seguimento": agg_top_value,
        }).reset_index()
        _.columns = ["Município", "Quantidade de empresas", "Principal atividade", "Principal seguimento"]
        _ = pd.merge(repo_geodf, _, on="Município", how="right")
        _
        
    elif index_file == 2:
        _ = parse_city_name(_, "municipio")
        _ = _.groupby("municipio").agg({
            "razao_social": "count",
            "atividade_principal": agg_top_value,
            "porte_da_empresa": agg_top_value,
        }).reset_index()
        for city in _.municipio:
            lat_long_repo.add(city)
        _.columns = ["Município", "Quantidade de empresas", "Principal atividade", "Principal porte"]
        _ = pd.merge(repo_geodf, _, on="Município", how="right")
        _
        
    elif index_file == 3:
        _ = parse_city_name(_, "municipio")
        _ = _.groupby("municipio").agg({
            "razao_social": "count",
            "nome_do_produto": agg_top_value,
            "quantidade_explorada": "sum",
            "tamanho_da_area": "sum",        
        }).reset_index()
        for city in _.municipio:
            lat_long_repo.add(city)
        _.columns = ["Município", "Quantidade de empresas", "Principal produto", "Quantidade explorada", "Quantidade de área"]
        _ = pd.merge(repo_geodf, _, on="Município", how="right")
        _

extracao_mineral.csv


Formato final do conjunto de dados.

In [330]:
print(_.info())
_

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   Município               16 non-null     object  
 1   geocodigo_              15 non-null     float64 
 2   nome_munic              15 non-null     object  
 3   uf                      15 non-null     object  
 4   geometry                15 non-null     geometry
 5   Quantidade de empresas  16 non-null     int64   
 6   Principal produto       16 non-null     object  
 7   Quantidade explorada    16 non-null     int64   
 8   Quantidade de área      16 non-null     int64   
dtypes: float64(1), geometry(1), int64(3), object(4)
memory usage: 1.3+ KB
None


,Município,geocodigo_,nome_munic,uf,geometry,Quantidade de empresas,Principal produto,Quantidade explorada,Quantidade de área
0,ANTONIO ALMEIDA,2200806.0,ANTÔNIO ALMEIDA,pi,MULTIPOINT (-44.19653 -7.21936),26,Calcário,1141751,898
1,AROEIRAS DO ITAIM,2200954.0,AROEIRAS DO ITAIM,pi,MULTIPOINT (-41.56320 -7.27874),1,tijolos e lajotas,300,360
2,CAMPO GRANDE DO PIAUI,2202133.0,CAMPO GRANDE DO PIAUÍ,pi,MULTIPOINT (-41.03120 -7.13599),2,argila,600,2
3,CAMPO MAIOR,2202208.0,CAMPO MAIOR,pi,MULTIPOINT (-42.16674 -4.82911),1,argila,10488,50
4,CAPITAO GERVASIO OLIVEIRA,2202455.0,CAPITÃO GERVÁSIO OLIVEIRA,pi,MULTIPOINT (-41.81612 -8.49684),2,Minério de níquel am,11945,2
5,CASTELO DO PIAUI,2202604.0,CASTELO DO PIAUÍ,pi,MULTIPOINT (-41.54732 -5.32154),10,quartzito ornamental,70515,123
6,FLORIANO,2203909.0,FLORIANO,pi,MULTIPOINT (-43.02065 -6.76967),7,areia,19540,152
7,FRONTEIRAS,2204303.0,FRONTEIRAS,pi,MULTIPOINT (-40.61723 -7.08841),7,Gnaisse,811836,8838
8,JOSE DE FREITAS,2205508.0,JOSÉ DE FREITAS,pi,MULTIPOINT (-42.57838 -4.76305),1,minerais,1200,3
9,JUAZEIRO DO PIAUI,2205516.0,JUAZEIRO DO PIAUÍ,pi,MULTIPOINT (-41.70260 -5.16137),3,Quartzito,16500,90


In [331]:
_.describe(include="all")

,Município,geocodigo_,nome_munic,uf,geometry,Quantidade de empresas,Principal produto,Quantidade explorada,Quantidade de área
count,16,1.500000e+01,15,15,15,16.000000,16,1.600000e+01,16.000000
unique,16,NaN,15,1,15,NaN,15,NaN,NaN
top,ANTONIO ALMEIDA,NaN,ANTÔNIO ALMEIDA,pi,MULTIPOINT (-44.196535 -7.219356),NaN,argila,NaN,NaN
freq,1,NaN,1,15,1,NaN,2,NaN,NaN
mean,NaN,2.204803e+06,NaN,NaN,NaN,7.875000,NaN,2.262582e+05,1306.812500
std,NaN,3.052345e+03,NaN,NaN,NaN,8.523497,NaN,4.142611e+05,2458.079826
min,NaN,2.200806e+06,NaN,NaN,NaN,1.000000,NaN,3.000000e+02,2.000000
25%,NaN,2.202332e+06,NaN,NaN,NaN,1.750000,NaN,6.130250e+03,50.000000
50%,NaN,2.204303e+06,NaN,NaN,NaN,5.000000,NaN,1.545000e+04,137.500000
75%,NaN,2.206893e+06,NaN,NaN,NaN,10.250000,NaN,1.668005e+05,1146.000000


salvando arquivo

In [332]:
_.to_file(PATH_TO_MAKE + "final/" + f"new_{files_to_make[index_file].replace('.csv', '.json')}", index=False, sep=",", encoding="utf-8")